In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import csv
from dragon import dragonV
import os

# data_parsing

In [2]:
root_path = '/Users/ivory/Documents/github/Philadelphia/real_final_data/'
subject_list = ['pp01', 'pp02', 'pp009', 'pp085', 'pp086', 'pp087', 'pp088', 'pp089'] 
#front, rear 2가지 존재. 파일명 format : {gait_catergory}_{front || rear}.xlsx
gait_category_list = ['gait1', 'gait2', 'fast', 'preferred', 'reaction', 'slow', 'stroop', 'turn']
seleted_openpose_joint_idx_list = [8, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24]

## xlsx -> nomalize -> lower joint

In [11]:
for each_subject in subject_list:
    #root/ppxx/
    each_subject_path = root_path + each_subject + '/'
    files = os.listdir(each_subject_path)

    ###xlsx_list###
    excel_name_list = [file for file in files if file.endswith('.xlsx')]
    
    ###gt_list####
    tmp = os.listdir(each_subject_path + 'gt/')
    gt_name_list = [file for file in tmp if file.endswith('.npy')]

    for each_excel in excel_name_list:
        ##xlsx -> list##
        frame_data_list = dragonV.xlsx2data(each_subject_path + each_excel)
        #print(f'{each_excel} : {len(frame_data_list[0])}')
        ##list -> normalize##
        norm_frame_data_list = dragonV.nomalize_data(frame_data_list)
        #print(f'{each_subject}->{each_excel} : {len(norm_frame_data_list[0])}')

        ## 여기까진 왔음...

        ##noramlize -> select lower joint pos##
        #seleted_norm_frame_data_list = dragonV.get_selected_joint_pos_frame_list(norm_frame_data_list, seleted_openpose_joint_idx_list)
        #print(f'{each_subject}->{each_excel} : {len(norm_frame_data_list[0])}')
        ##frame data : gt pair##


TypeError: list indices must be integers or slices, not list

# data_preprocessing

outputs, _status = cell(output)<br>

(모든 timesteps에 대한 결과)<br>
outputs -> [bactch x timesteps x output]<br><br>
(마지막 timestpes에 대한 결과)<br>
_status -> [1 x batch x output]<br>

## network 정의

In [49]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.linear(hidden_size, output_size,)

    def forward(self, x):
        outputs, _ = self.lstm(x)
        #out : batch x timesteps x output_size
        out = self.fc(out[: -1, :])
        return torch.sigmoid(out)

## hyper-parameter 정의

In [ ]:
# batch x timesteps x input_size -> ? x 9 x 26
input_size  = 26
hidden_size = 128
output_size = 4
learning_rate = 0.001 
batch_size = 4

epoch_size = 1000

## criterian && optimizer 정의

In [ ]:
net = LSTM(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate)

## training network

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [55]:
def train_network(model, train_loader, num_epochs, criterion, optimizer):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    if(epoch % 100 == 0):
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [ ]:
def evaluate(model, test_loader, criterion, threshold):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    accuracy = 0.0
    precision = 0.0
    recall = 0.0
    f1 = 0.0

    test_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            


    # 다중 레이블 분류 평가 지표 계산, macro : 각 클래스에 대해 개별적으로 평가
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f'Average Test Loss: {avg_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')